In [10]:
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sys

In [9]:
metric = 'cpu.usage.average'
vm = 'DataLake-DBN1'

df = df[df['vm']==vm]
df = df[df['metric']==metric]

df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%d %h:%m:%s")
df.sort_values(by=['timestamp'], inplace=True)
df = df.rename(columns={'timestamp':'ds', 'value': 'y'})
df

,vm,metric,y,ds
0,DataLake-DBN1,cpu.usage.average,5.11,2025-11-25 17:00:00
16,DataLake-DBN1,cpu.usage.average,5.03,2025-11-25 17:30:00
32,DataLake-DBN1,cpu.usage.average,4.77,2025-11-25 18:00:00
48,DataLake-DBN1,cpu.usage.average,4.68,2025-11-25 18:30:00
64,DataLake-DBN1,cpu.usage.average,4.63,2025-11-25 19:00:00
...,...,...,...,...
4752,DataLake-DBN1,cpu.usage.average,5.38,2025-12-01 21:30:00
4768,DataLake-DBN1,cpu.usage.average,5.08,2025-12-01 22:00:00
4784,DataLake-DBN1,cpu.usage.average,5.13,2025-12-01 22:30:00
4800,DataLake-DBN1,cpu.usage.average,5.11,2025-12-01 23:00:00


In [11]:
# Выгружаем из базы данных
# Конфигурация
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'database': 'server_metrics',
    'user': 'postgres',
    'password': 'postgres'
}

In [12]:
def db_export(vm, metric, start_date, end_date):
    """
    Выгрузка данных за период времени

    Args:
        vm: сервер
        metric: метрика
        start_date: выгрузка С
        end_date: выгрузка ПО
    """

    try:
        # Подключение к базе
        conn = psycopg2.connect(**DB_CONFIG)

        # SQL запрос
        query = """
        SELECT
            timestamp,
            value
        FROM server_metrics_fact
        WHERE
            vm = %s
            AND metric = %s
            AND timestamp BETWEEN %s AND %s
        ORDER BY timestamp ASC
        """

        # Выполнение запроса
        df = pd.read_sql_query(query, conn, params=(vm, metric, start_date, end_date))

        # Закрытие соединения
        conn.close()

        if df.empty:
            print(f"Данные не найдены для {vm} - {metric}")
            return None

        # Вывод информации
        print(f"Выгружено {len(df)} записей")
        print(f"Период: {df['timestamp'].min()} - {df['timestamp'].max()}")
        print(f"Значения: {df['value'].min():.2f}% - {df['value'].max():.2f}%")
        print(f"Среднее: {df['value'].mean():.2f}%")

        return df

    except Exception as e:
        print(f"Ошибка: {e}")
        return None

In [13]:
# Пример использования
data = db_export("DataLake-DBN1", "cpu.usage.average", '2025-11-26 01:00:00', '2025-11-28 01:00:00')
data

Выгружено 97 записей
Ошибка: No module named 'numpy.rec'


/var/folders/n6/9hwnx40x1g53w79wdp1sqz5w0000gn/T/ipykernel_64482/3397119797.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(vm, metric, start_date, end_date))


In [6]:
critical_metrics = ['cpu.usage.average', 'cpu.ready.summation', 'mem.usage.average', 'disk.usage.average']

In [7]:
# Посомтрим как работает модель для пары сервер метрика на данных за 6 дней

In [3]:
import pandas as pd

In [4]:
df_train = pd.read_excel('/Users/sweetd0ve/dashboard/data/processed/DataLake-DBN1_cpu.usage.average_2025-11-25 17:00:00_2025-11-30 23:30:00.xlsx')
df_train

,vm,metric,value,timestamp
0,DataLake-DBN1,cpu.usage.average,5.11,2025-11-25 17:00:00
1,DataLake-DBN1,cpu.usage.average,5.03,2025-11-25 17:30:00
2,DataLake-DBN1,cpu.usage.average,4.77,2025-11-25 18:00:00
3,DataLake-DBN1,cpu.usage.average,4.68,2025-11-25 18:30:00
4,DataLake-DBN1,cpu.usage.average,4.63,2025-11-25 19:00:00
...,...,...,...,...
249,DataLake-DBN1,cpu.usage.average,5.14,2025-11-30 21:30:00
250,DataLake-DBN1,cpu.usage.average,5.18,2025-11-30 22:00:00
251,DataLake-DBN1,cpu.usage.average,5.24,2025-11-30 22:30:00
252,DataLake-DBN1,cpu.usage.average,5.19,2025-11-30 23:00:00


In [5]:
df_test = pd.read_excel('/Users/sweetd0ve/dashboard/data/processed/DataLake-DBN1_cpu.usage.average_2025-12-01 00:00:00_2025-12-01 23:30:00.xlsx')
df_test

,vm,metric,value,timestamp
0,DataLake-DBN1,cpu.usage.average,5.18,2025-12-01 00:00:00
1,DataLake-DBN1,cpu.usage.average,5.10,2025-12-01 00:30:00
2,DataLake-DBN1,cpu.usage.average,5.23,2025-12-01 01:00:00
3,DataLake-DBN1,cpu.usage.average,11.34,2025-12-01 01:30:00
4,DataLake-DBN1,cpu.usage.average,5.18,2025-12-01 02:00:00
5,DataLake-DBN1,cpu.usage.average,5.12,2025-12-01 02:30:00
6,DataLake-DBN1,cpu.usage.average,5.21,2025-12-01 03:00:00
7,DataLake-DBN1,cpu.usage.average,5.00,2025-12-01 03:30:00
8,DataLake-DBN1,cpu.usage.average,5.03,2025-12-01 04:00:00
9,DataLake-DBN1,cpu.usage.average,5.07,2025-12-01 04:30:00


In [3]:
!pip install prophet

  Using cached prophet-1.2.1-py3-none-macosx_10_11_x86_64.whl (12.7 MB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_x86_64.whl (6.9 MB)
  Using cached matplotlib-3.9.4-cp39-cp39-macosx_10_12_x86_64.whl (7.9 MB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
  Using cached pandas-2.3.3-cp39-cp39-macosx_10_9_x86_64.whl (11.6 MB)
  Using cached cmdstanpy-1.3.0-py3-none-any.whl (99 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached holidays-0.83-py3-none-any.whl (1.3 MB)
  Using cached stanio-0.5.1-py3-none-any.whl (8.1 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached fonttools-4.60.2-cp39-cp39-macosx_10_9_x86_64.whl (2.4 MB)
  Using cached pyparsing-3.3.1-py3-none-any.whl (121 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-macosx_10_9_x86_64.whl (65 kB)
  Using cached contourpy-1.3.0-cp39-cp39-macosx_10_9_x86_64.whl (265 kB)
  Using cached pillow-11.3.0-cp39-cp39-macosx_10_10_x86_64.whl (5.3 MB)
  Using cached pytz-2025.2

In [4]:
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [5]:
?Prophet